In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime, timedelta
import pickle

In [2]:
files = [
    'DrDoS_NTP.csv',
    'DrDoS_DNS.csv',
    'DrDoS_LDAP.csv',
    'DrDoS_MSSQL.csv',
    'DrDoS_NetBIOS.csv',
    'DrDoS_SNMP.csv',
    'DrDoS_SSDP.csv',
    'DrDoS_UDP.csv',
    'UDPLag.csv',
    'Syn.csv',
    'TFTP.csv',
]

## Only NTP

In [5]:
data = pd.read_csv(f'../data/raw/csv/01-12/{files[0]}')
data.columns = [x.strip() for x in data.columns]
print(data.shape)
data.head()

/home/caiolmart/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1217007, 88)


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,0,172.16.0.5-192.168.50.1-60675-80-6,172.16.0.5,60675,192.168.50.1,80,6,2018-12-01 09:17:11.183810,5220876,12,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,205.174.165.72/c.php,1,DrDoS_NTP
1,7,172.16.0.5-192.168.50.1-60676-80-6,172.16.0.5,60676,192.168.50.1,80,6,2018-12-01 09:17:11.205636,12644252,5,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_NTP
2,12858,192.168.50.7-65.55.163.78-50458-443-6,65.55.163.78,443,192.168.50.7,50458,6,2018-12-01 09:17:12.634569,3,2,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,BENIGN
3,10191,192.168.50.7-65.55.163.78-50465-443-6,65.55.163.78,443,192.168.50.7,50465,6,2018-12-01 09:17:13.458370,3,2,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,BENIGN
4,239,192.168.50.253-224.0.0.5-0-0-0,192.168.50.253,0,224.0.0.5,0,0,2018-12-01 09:17:13.470913,114329232,52,...,2.466441,15.0,6.0,9527428.0,248706.681286,9950741.0,9092248.0,0,0,BENIGN


In [25]:
group = data.groupby(['Source IP', 'Destination IP', 'Label']).agg({'Timestamp': ['min', 'max']}).reset_index()
print(group.shape)
group.head()

(1118, 5)


Source IP   Destination IP   Label                   Timestamp  \
                                                                  min   
0        0.0.0.0  255.255.255.255  BENIGN  2018-12-01 09:18:30.010736   
1  104.117.0.160     192.168.50.7  BENIGN  2018-12-01 10:04:27.720354   
2  104.118.6.139     192.168.50.7  BENIGN  2018-12-01 10:17:04.647026   
3  104.18.58.178     192.168.50.7  BENIGN  2018-12-01 10:14:09.655233   
4  104.18.58.178     192.168.50.8  BENIGN  2018-12-01 09:43:32.899697   

                               
                          max  
0  2018-12-01 10:48:30.146543  
1  2018-12-01 10:04:27.720354  
2  2018-12-01 10:17:04.647026  
3  2018-12-01 10:14:55.627975  
4  2018-12-01 09:43:32.899697

In [27]:
group[group.Label == 'DrDoS_NTP']

Source IP Destination IP      Label                   Timestamp  \
                                                                    min   
30     172.16.0.5   192.168.50.1  DrDoS_NTP  2018-12-01 09:17:11.183810   
248  192.168.50.1     172.16.0.5  DrDoS_NTP  2018-12-01 09:17:16.404689   

                                 
                            max  
30   2018-12-01 10:51:39.778786  
248  2018-12-01 10:51:38.825113

## Only DNS

In [28]:
data = pd.read_csv(f'../data/raw/csv/01-12/{files[1]}')
data.columns = [x.strip() for x in data.columns]
print(data.shape)
data.head()

/home/caiolmart/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(5074413, 88)


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,425,172.16.0.5-192.168.50.1-634-60495-17,172.16.0.5,634,192.168.50.1,60495,17,2018-12-01 10:51:39.813448,28415,97,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS
1,430,172.16.0.5-192.168.50.1-60495-634-17,192.168.50.1,634,172.16.0.5,60495,17,2018-12-01 10:51:39.820842,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,DrDoS_DNS
2,1654,172.16.0.5-192.168.50.1-634-46391-17,172.16.0.5,634,192.168.50.1,46391,17,2018-12-01 10:51:39.852499,48549,200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS
3,2927,172.16.0.5-192.168.50.1-634-11894-17,172.16.0.5,634,192.168.50.1,11894,17,2018-12-01 10:51:39.890213,48337,200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS
4,694,172.16.0.5-192.168.50.1-634-27878-17,172.16.0.5,634,192.168.50.1,27878,17,2018-12-01 10:51:39.941151,32026,200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS


In [51]:
group = data.groupby(['Source IP', 'Destination IP', 'Label']).agg({'Timestamp': 'max'}).reset_index()
print(group.shape)
group.head()

(401, 4)


,Source IP,Destination IP,Label,Timestamp
0,0.0.0.0,255.255.255.255,BENIGN,2018-12-01 11:20:44.974670
1,104.36.115.113,192.168.50.6,BENIGN,2018-12-01 11:21:16.705372
2,104.36.115.113,192.168.50.8,BENIGN,2018-12-01 11:04:05.120243
3,104.88.29.90,192.168.50.6,BENIGN,2018-12-01 11:02:37.680624
4,104.88.46.113,192.168.50.7,BENIGN,2018-12-01 11:15:31.507846


In [40]:
group[group.Label == 'DrDoS_DNS']

,Source IP,Destination IP,Label,Timestamp
,,,,max
18,172.16.0.5,192.168.50.1,DrDoS_DNS,2018-12-01 11:22:40.254719
117,192.168.50.1,172.16.0.5,DrDoS_DNS,2018-12-01 11:22:39.740027


## Create a label dictionary

It will have the following structure:

It is a dictionary with: {
  - (Source IP, Destination IP): {
    - Label : Last Timestamp
  
  }

}

I will run here only for the first 4 attacks, since my RAM does not support all tables.

In [6]:
label_dict = defaultdict(dict)
for file in files[:4]:
    print(f'Reading {file}')
    data = pd.read_csv(f'../data/raw/csv/01-12/{file}')
    data.columns = [x.strip() for x in data.columns]
    print(f'Preparing timestamps')
    data['timestamp'] = data['Timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f') + timedelta(hours=4))
    print(data.shape)
    data.head()
    print(f'Grouping data')
    group = data.groupby(['Source IP', 'Destination IP', 'Label']).agg({'Timestamp': 'max'}).reset_index()
    print(f'Inserting into dictionary')
    for index, row in group.iterrows():
        label_dict[(row['Source IP'], row['Destination IP'])][row['Label']] = row['Timestamp']
    del data
    del group
    print('Done\n')

Reading DrDoS_NTP.csv
Preparing timestamps
(1217007, 89)
Grouping data
Inserting into dictionary
Done

Reading DrDoS_DNS.csv
Preparing timestamps
(5074413, 89)
Grouping data
Inserting into dictionary
Done

Reading DrDoS_LDAP.csv
Preparing timestamps
(2181542, 89)
Grouping data
Inserting into dictionary
Done

Reading DrDoS_MSSQL.csv
Preparing timestamps
(4524498, 89)
Grouping data
Inserting into dictionary
Done



In [8]:
len(label_dict)

1338

## Reading Label Dictionary

### Train

In [10]:
with open('../data/interim/part_c/train_label_dict.pkl', 'rb') as filename:
    label_dict = pickle.load(filename)

In [16]:
len(label_dict)

2043

As we can see below, all attacks come from the same IP address (172.16.0.5)

In [20]:
{k: v for k, v in label_dict.items() if 'BENIGN' not in v}

{('172.16.0.5', '192.168.50.1'): {'DrDoS_NTP': '2018-12-01 10:51:39.778786',
  'DrDoS_DNS': '2018-12-01 11:22:40.254719',
  'DrDoS_LDAP': '2018-12-01 11:32:32.915361',
  'DrDoS_MSSQL': '2018-12-01 11:47:08.463107',
  'DrDoS_NetBIOS': '2018-12-01 12:00:13.902732',
  'DrDoS_SNMP': '2018-12-01 12:23:13.663368',
  'DrDoS_SSDP': '2018-12-01 12:36:57.627789',
  'DrDoS_UDP': '2018-12-01 13:04:45.928382',
  'UDP-lag': '2018-12-01 13:30:30.740425',
  'WebDDoS': '2018-12-01 13:30:30.664221',
  'Syn': '2018-12-01 13:34:27.403143',
  'TFTP': '2018-12-01 17:16:38.374479'},
 ('192.168.50.1', '172.16.0.5'): {'DrDoS_NTP': '2018-12-01 10:51:38.825113',
  'DrDoS_DNS': '2018-12-01 11:22:39.740027',
  'DrDoS_LDAP': '2018-12-01 11:32:32.723493',
  'DrDoS_MSSQL': '2018-12-01 11:47:07.499031',
  'DrDoS_NetBIOS': '2018-12-01 12:00:13.662425',
  'DrDoS_SNMP': '2018-12-01 12:23:12.850532',
  'DrDoS_SSDP': '2018-12-01 12:36:56.858758',
  'DrDoS_UDP': '2018-12-01 13:04:45.170823',
  'UDP-lag': '2018-12-01 13:30:1

In [21]:
{k: v for k, v in label_dict.items() if len(v) > 1}

{('172.16.0.5', '192.168.50.1'): {'DrDoS_NTP': '2018-12-01 10:51:39.778786',
  'DrDoS_DNS': '2018-12-01 11:22:40.254719',
  'DrDoS_LDAP': '2018-12-01 11:32:32.915361',
  'DrDoS_MSSQL': '2018-12-01 11:47:08.463107',
  'DrDoS_NetBIOS': '2018-12-01 12:00:13.902732',
  'DrDoS_SNMP': '2018-12-01 12:23:13.663368',
  'DrDoS_SSDP': '2018-12-01 12:36:57.627789',
  'DrDoS_UDP': '2018-12-01 13:04:45.928382',
  'UDP-lag': '2018-12-01 13:30:30.740425',
  'WebDDoS': '2018-12-01 13:30:30.664221',
  'Syn': '2018-12-01 13:34:27.403143',
  'TFTP': '2018-12-01 17:16:38.374479'},
 ('192.168.50.1', '172.16.0.5'): {'DrDoS_NTP': '2018-12-01 10:51:38.825113',
  'DrDoS_DNS': '2018-12-01 11:22:39.740027',
  'DrDoS_LDAP': '2018-12-01 11:32:32.723493',
  'DrDoS_MSSQL': '2018-12-01 11:47:07.499031',
  'DrDoS_NetBIOS': '2018-12-01 12:00:13.662425',
  'DrDoS_SNMP': '2018-12-01 12:23:12.850532',
  'DrDoS_SSDP': '2018-12-01 12:36:56.858758',
  'DrDoS_UDP': '2018-12-01 13:04:45.170823',
  'UDP-lag': '2018-12-01 13:30:1

### Test

In [22]:
with open('../data/interim/part_c/test_label_dict.pkl', 'rb') as filename:
    label_dict = pickle.load(filename)

In [23]:
len(label_dict)

3233

As we can see below, all attacks come from the same IP address (172.16.0.5)

In [24]:
{k: v for k, v in label_dict.items() if 'BENIGN' not in v}

{('172.16.0.5', '192.168.50.4'): {'Portmap': '2018-11-03 10:01:48.920515',
  'NetBIOS': '2018-11-03 10:18:39.121734',
  'LDAP': '2018-11-03 10:31:59.708498',
  'MSSQL': '2018-11-03 10:51:59.988466',
  'UDP': '2018-11-03 11:12:58.054429',
  'Syn': '2018-11-03 17:36:41.528982',
  'UDPLag': '2018-11-03 11:27:53.999014'},
 ('192.168.50.4', '172.16.0.5'): {'Portmap': '2018-11-03 10:01:48.767479',
  'NetBIOS': '2018-11-03 10:18:28.139718',
  'LDAP': '2018-11-03 10:31:59.708671',
  'MSSQL': '2018-11-03 10:51:59.908375',
  'UDP': '2018-11-03 11:12:58.054529',
  'Syn': '2018-11-03 17:36:22.391171',
  'UDPLag': '2018-11-03 11:27:53.999196'}}

In [25]:
{k: v for k, v in label_dict.items() if len(v) > 1}

{('172.16.0.5', '192.168.50.4'): {'Portmap': '2018-11-03 10:01:48.920515',
  'NetBIOS': '2018-11-03 10:18:39.121734',
  'LDAP': '2018-11-03 10:31:59.708498',
  'MSSQL': '2018-11-03 10:51:59.988466',
  'UDP': '2018-11-03 11:12:58.054429',
  'Syn': '2018-11-03 17:36:41.528982',
  'UDPLag': '2018-11-03 11:27:53.999014'},
 ('192.168.50.4', '172.16.0.5'): {'Portmap': '2018-11-03 10:01:48.767479',
  'NetBIOS': '2018-11-03 10:18:28.139718',
  'LDAP': '2018-11-03 10:31:59.708671',
  'MSSQL': '2018-11-03 10:51:59.908375',
  'UDP': '2018-11-03 11:12:58.054529',
  'Syn': '2018-11-03 17:36:22.391171',
  'UDPLag': '2018-11-03 11:27:53.999196'}}